In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-training.csv',index_col=0)
test = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-test.csv',index_col=0)
sample = pd.read_csv('/kaggle/input/GiveMeSomeCredit/sampleEntry.csv')

train.shape, test.shape, sample.shape

In [ ]:
train.isnull().sum(), '*'*50, test.isnull().sum()

In [ ]:
#假设训练集和验证集分布一致，则可以用两者之和计算数据特征：中位数 平均数 众数
#考虑到现实情况并不能一次知道test的属性，所以不用
#但如果把所有数据看成行业经验，可以学习得到总的统计量

X_all = pd.concat([train,test], axis=0)
X_all.shape

In [ ]:
# train['MonthlyIncome'].astype('float64')
default_dict = {} 
for attr in ['MonthlyIncome', 'NumberOfDependents']:
    print(attr, end='\t')
    median, mean, mode = X_all[attr].median(), X_all[attr].mean(), X_all[attr].mode()
    mode = mode.values
    if len(mode) == 1:
        mode = mode[0]
    default_dict[attr] = [median, mean, mode]
    print(median, mean, mode)
default_dict

In [ ]:
# median中位数 填充 月收入
# 平均数 填充 NumberOfDependents

In [ ]:
train.MonthlyIncome.fillna(default_dict['MonthlyIncome'][0], inplace=True)
test.MonthlyIncome.fillna(default_dict['MonthlyIncome'][0], inplace=True)
train.NumberOfDependents.fillna(default_dict['NumberOfDependents'][1], inplace=True)
test.NumberOfDependents.fillna(default_dict['NumberOfDependents'][1], inplace=True)

train.isnull().sum(), '*'*50, test.isnull().sum()

In [ ]:
#暂时不做其他的数据处理
#直接load 模型， 进行训练预测分类

In [ ]:
#先分割数据集为X_train, y_train, X_test
X_train = train.iloc[:,1:].values
y_train = train.iloc[:,0].values
X_test = test.iloc[:,1:].values
X_train.shape, y_train.shape, X_test.shape

15万训练数据 \
10万1503个测试数据

In [ ]:
#在训练之前先归一化,
# https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling

from sklearn import preprocessing

train_scaler = preprocessing.StandardScaler().fit(X_train)
print( train_scaler.mean_ , '\n'+'-'*50+'\n', train_scaler.scale_)
print('='*50)
test_scaler = preprocessing.StandardScaler().fit(X_test)
print( test_scaler.mean_ , '\n'+'-'*50+'\n', test_scaler.scale_)



可见在两个数据集上属性分布还是有一定的区别，\
所以我选择分别归一化

In [ ]:
#分别归一化并查看结果
X_train_scaled = train_scaler.transform(X_train)
X_test_scaled = test_scaler.transform(X_test)

X_train_scaled.mean(axis=0), X_train_scaled.std(axis=0),   X_test_scaled.mean(axis=0), X_test_scaled.std(axis=0)


In [ ]:
from sklearn.linear_model import LogisticRegression as LR

rgs = LR()
rgs.fit(X_train_scaled, y_train)
y_pred = rgs.predict(X_test_scaled) 
y_pred_proba = rgs.predict_proba(X_test_scaled)
y_pred, y_pred_proba
#pred就是取 pred_proba中较大的类标签为最终结果

In [ ]:
y_probs = y_pred_proba[:,1]
y_probs

In [ ]:
sample.Probability = y_probs
sample.to_csv('./LR-03.csv',index=False)
sample

In [ ]:
reload = pd.read_csv('LR-03.csv')
reload